In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
import random
import nltk
import json
import pandas as pd

In [2]:
intents_file = open('intents.json', encoding='utf-8').read()
intents = json.loads(intents_file)

In [3]:
intentions = intents['intentions']

tags = []
questions = []
reponses = {}

for intention in intentions:
    tag = intention.get("tag")
    for question in intention.get("questions"):
        questions.append(question)
        tags.append(tag)
    reponses[tag] = intention.get("reponses")


In [27]:
df = pd.DataFrame({
    "questions": questions,
    "label": tags
})

df

,questions,label
0,Bonjour,salutations
1,Bonsoir,salutations
2,Salut,salutations
3,Comment vas-tu?,salutations
4,Comment ça va?,salutations
...,...,...
98,Quel est votre adresse mail,contacts
99,Quel est votre numéro de téléphone,contacts
100,Quels sont vos contacts,contacts
101,Comment vous contacter,contacts


In [5]:
import string
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer("french")

def replace_punct_with_space(text):
    punctuations = string.punctuation
    for punct in punctuations:
        text = text.replace(punct, ' ')
    return text

def sans_accent(text):
    text = text.replace("é","e")
    text = text.replace("è","e")
    text = text.replace("ê","e")
    text = text.replace("à","a")
    text = text.replace("ç","c")
    return text

def tokenize(text):
    return nltk.tokenize.word_tokenize(text)

def stemming(tokenized_text):
    text=[stemmer.stem(word.lower())  for word in tokenized_text]
    return text

def text_preprocess(text):
    text = replace_punct_with_space(text)
    text = sans_accent(text)
    tokenized_text = tokenize(text)
    text_preprocessed = " ".join(stemming(tokenized_text))
    return text_preprocessed

df["questions"] = df["questions"].apply(lambda x: text_preprocess(x))

In [6]:
df

,questions,label
0,bonjour,salutations
1,bonsoir,salutations
2,salut,salutations
3,comment vas tu,salutations
4,comment ca va,salutations
...,...,...
98,quel est votr adress mail,contacts
99,quel est votr numero de telephon,contacts
100,quel sont vos contact,contacts
101,comment vous contact,contacts


In [7]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(df["questions"])

data_vect = vectorizer.transform(df["questions"])

In [8]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(df["label"])
len(set(encoded_labels))

21

In [9]:
#nltk.download('punkt')

In [10]:
#nltk.download('omw-1.4')

In [11]:
len(data_vect.toarray()), len(encoded_labels)

(103, 103)

In [28]:
len(data_vect.toarray()[0])

140

In [13]:
model = Sequential()
model.add(Dense(128, input_shape=(len(data_vect.toarray()[0]),), activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(set(encoded_labels)), activation='softmax'))
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='sparse_categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [14]:
model.fit(data_vect.toarray(), encoded_labels, epochs=1000, batch_size=80)

Epoch 1/1000
2/2 [==============================] - 1s 13ms/step - loss: 3.1064 - accuracy: 0.0194
Epoch 2/1000
2/2 [==============================] - 0s 7ms/step - loss: 3.1038 - accuracy: 0.0291
Epoch 3/1000
2/2 [==============================] - 0s 6ms/step - loss: 3.0480 - accuracy: 0.0291
Epoch 4/1000
2/2 [==============================] - 0s 6ms/step - loss: 3.0733 - accuracy: 0.0388
Epoch 5/1000
2/2 [==============================] - 0s 6ms/step - loss: 3.0143 - accuracy: 0.0485
Epoch 6/1000
2/2 [==============================] - 0s 5ms/step - loss: 2.9819 - accuracy: 0.0874
Epoch 7/1000
2/2 [==============================] - 0s 5ms/step - loss: 3.0233 - accuracy: 0.0194
Epoch 8/1000
2/2 [==============================] - 0s 5ms/step - loss: 2.9869 - accuracy: 0.0874
Epoch 9/1000
2/2 [==============================] - 0s 6ms/step - loss: 2.9643 - accuracy: 0.0583
Epoch 10/1000
2/2 [==============================] - 0s 6ms/step - loss: 2.9474 - accuracy: 0.1165
Epoch 11/1000
2/2 

In [15]:
def input_preprocess(text):
    text = text_preprocess(text)
    text_vect = vectorizer.transform([text])
    return text_vect.toarray()

In [29]:
msg = "quels sont les formations que vous proposées"
input = input_preprocess(msg)
pred = np.argmax(model.predict(input))
decoded_data = encoder.inverse_transform([pred])
tag = decoded_data[0]
random.choice(reponses.get(tag))

1/1 [==============================] - 0s 57ms/step


'DIT est le lieu idéal pour les étudiants souhaitant se spécialiser dans les domaines de la science des données, du big data et de l\'intelligence artificielle, avec des formations telles que la Data Scientist Intensive, la Licence Informatique "Big Data", le Master "Intelligence Artificielle" ainsi que d\'autres formations certifiantes pour développer les compétences clés et les opportunités professionnelles.'